# Creating a web-app

* To demonstrate what sorts of information will be provided to users, I build a quick web app using streamlit below. 
* In this illustrative example, I take "Andy Zipper" as a user (i.e., sender) and give four different "recipient" options. 

## 1. Loading libraries

In [231]:
import streamlit as st

import numpy as np
import pandas as pd
import re, os
from ast import literal_eval

## 2. Setting a title

In [232]:
st.title('-B-R-I-D-G-E-M-A-I-L-')

## 3. Importing input data

In [432]:
# import data
#email_path = 'C:/Users/Margeum/Dropbox/DS projects/05. Email data/emails_in_csv'  # Place your directory path here
#os.chdir(email_path)
pw_path_length_df = pd.read_csv('./pw_path_length_df.csv')
address_user_df = pd.read_csv('./address_user_df.csv')


# We are using Andy Zipper's account as an example--her user names is "zipper-a"
user_name = 'zipper-a'


# Andy Zipper's stranger scores dataframe
stranger_df = pw_path_length_df[pw_path_length_df['node_i']=="andy.zipper@enron.com"]
stranger_df = stranger_df[['node_j', 's_path_length']]


# Import the user's bag of words dataset
user_bow_file_name = './' + user_name + '_nonoverlap.pkl'
bow_df = pd.read_pickle(user_bow_file_name)


# Loading Andy Zipper's "To" addresses list
# Currnetly, I am displaying 4 recipients for the demonstration purpose
# For the complete emailing list: #node_j_list = list(stranger_df['node_j']) 
node_j_list = ["phillip.allen@enron.com", "jeffrey.hodge@enron.com",  
              "larry.campbell@enron.com", "craig.dean@enron.com"]    # Might add: "kate.symes@enron.com"
empty_str = ''
node_j_list.insert(0,empty_str)   # "To" addresses to be displayed in the app 


# Andy Zipper's and the 4 recipients' word-meaning dataframe
node_i_dict_file = str(user_name) + '-meaning-example.pkl'
node_i_dict = pd.read_pickle(node_i_dict_file)
# Making a list of keywords (currently, 100 keywords are used)
keywords_list = list(set(node_i_dict['keyword']))

## 4. Taking user inputs and returning relevant information

In [413]:
# Let the user choose a recipient from a dropdown menu
option = st.selectbox(
    "To:", node_j_list)

# Add text area
txt = st.text_area('Content:', height = 100)


# Stranger detector section
if option: 
   
    st.header("Stranger detector")
    
    stranger_score = int(stranger_df['s_path_length'][stranger_df['node_j']== option])
    st.write('Your stranger score: ' + str(stranger_score) + ' out of 5') 

    
    # Jargon detector section and meaning section for 'distant' pairs 
    if (stranger_score > 2):
        
        if txt: 

            st.header("Jargon detector")

            user_jargon = []
            for item in list(re.split(r'[,.\s]\s*', txt)):
                if item.lower() in list(bow_df[bow_df['node_j']==option]['non_overlap'])[0]: 
                    user_jargon.append(item) 

            jargon = user_jargon 
            if len(user_jargon) == 0: 
                st.write("Your email is jargon-free! Good job!")
            else:
                st.write("Your recipient does not use the following word(s). You might want to provide definition(s) and/or context(s)", list(jargon))


            st.header("Context detector")
            
            option_user_name = list(address_user_df[address_user_df['address'] == option]['user'])[0]

            to_compare_meaning = []

            for item in list(re.split(r'[,.?!\s]\s*', txt)):
                if item in keywords_list: 
                    to_compare_meaning.append(item) 

                    
            if len(to_compare_meaning) == 0: 
                st.write("We didn't find any words with potential meaning discrepancies.")
            else:
                
                st.write("You and your recipient might use the following words differently: " + str(to_compare_meaning))
                
                for k in range(len(to_compare_meaning)): 
                    
                    st.subheader("Potential gaps in using \""  + str(to_compare_meaning[k]) +"\"")    
                    
                    # Meanings in the user's space but not in the recipient's space (complement of the recipient meaning set)
                    meaning_i_j = list((set(list(node_i_dict.loc[(node_i_dict['user']==user_name)
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0]) -  
                                   set(list(node_i_dict.loc[(node_i_dict['user']==option_user_name) 
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0])))
                    
                    # Meanings in the recipient's space but not in the user's space (complement of the user meaning set)
                    meaning_j_i = list((set(list(node_i_dict.loc[(node_i_dict['user']==option_user_name) 
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0]) -  
                                        set(list(node_i_dict.loc[(node_i_dict['user']==user_name) 
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0])))

                    #Intersection of the two user's meaning spaces
                    meaning_intersect = list((set(list(node_i_dict.loc[(node_i_dict['user']==option_user_name) 
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0]) &  
                                        set(list(node_i_dict.loc[(node_i_dict['user']==user_name) 
                                              & (node_i_dict['keyword']==to_compare_meaning[k])]["high_att_words"])[0])))


#                    st.write("Below are the words highly associated with " + str(to_compare_meaning[k]))

#                    st.write("BOTH of you use it along with: " + 
#                          str(meaning_intersect))

                    st.write("You tend to use it with: " + 
                          str(meaning_i_j))

                    st.write("Your recipient tends to use with: " + str(meaning_j_i))


send = st.button('Send')
if send:
    st.write('Message sent')


